<a href="https://colab.research.google.com/github/sakisakichen/de-zoomcamp2024/blob/main/workshop1_Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [21]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 13
generator = square_root_generator(limit)

n= 0
sum = 0
for sqrt_value in generator:
    sum += sqrt_value
    n+= 1
    # print("n", n )
    # print("sqrt value " ,sqrt_value)
    # print("sum:" ,sum)

    print(f'{n}sqrt value{sqrt_value}\n sum{sum}')

1sqrt value1.0
 sum1.0
2sqrt value1.4142135623730951
 sum2.414213562373095
3sqrt value1.7320508075688772
 sum4.146264369941973
4sqrt value2.0
 sum6.146264369941973
5sqrt value2.23606797749979
 sum8.382332347441762
6sqrt value2.449489742783178
 sum10.83182209022494
7sqrt value2.6457513110645907
 sum13.47757340128953
8sqrt value2.8284271247461903
 sum16.30600052603572
9sqrt value3.0
 sum19.30600052603572
10sqrt value3.1622776601683795
 sum22.4682781862041
11sqrt value3.3166247903554
 sum25.7849029765595
12sqrt value3.4641016151377544
 sum29.249004591697254
13sqrt value3.605551275463989
 sum32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [22]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [25]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [35]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)



Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1712721080.057548 is LOADED and contains no failed jobs


In [36]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [37]:


# and the data

print("\n\n\n peple table below:")

the_people = conn.sql("SELECT * FROM people").df()
display(the_people)

print("the age sum:", the_people['age'].sum())





 peple table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712721080.057548,IXjM31Wd/nF9xA,None
1,2,Person_2,27,City_A,1712721080.057548,dtiBFnGqhp/egg,None
2,3,Person_3,28,City_A,1712721080.057548,R1CmdB/I3eBaWw,None
3,4,Person_4,29,City_A,1712721080.057548,6tuHQmimiJH1tA,None
4,5,Person_5,30,City_A,1712721080.057548,vmsDtQpCVsL75g,None


the age sum: 140


In [38]:
#  Second set of people
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1712721091.817935 is LOADED and contains no failed jobs


In [39]:
print("\n\n\n peple table below:")

the_people = conn.sql("SELECT * FROM people").df()
display(the_people)

print("the age sum:", the_people['age'].sum())




 peple table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712721080.057548,IXjM31Wd/nF9xA,None
1,2,Person_2,27,City_A,1712721080.057548,dtiBFnGqhp/egg,None
2,3,Person_3,28,City_A,1712721080.057548,R1CmdB/I3eBaWw,None
3,4,Person_4,29,City_A,1712721080.057548,6tuHQmimiJH1tA,None
4,5,Person_5,30,City_A,1712721080.057548,vmsDtQpCVsL75g,None
5,3,Person_3,33,City_B,1712721091.817935,3OVE8qxgLRQS/Q,Job_3
6,4,Person_4,34,City_B,1712721091.817935,zA9cQA/oT9b+ag,Job_4
7,5,Person_5,35,City_B,1712721091.817935,nYuyszvBDpgLSQ,Job_5
8,6,Person_6,36,City_B,1712721091.817935,GBlsnnbg8xxZ8A,Job_6
9,7,Person_7,37,City_B,1712721091.817935,Twh0LSNH+Mj2UQ,Job_7


the age sum: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [48]:
# CREATE A SECOND TABLE SO I CAN PRACTICE MERGING ON THE KEYS

# load people_1 generator to a new table
info = generators_pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1712721656.665147 is LOADED and contains no failed jobs


In [49]:
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘

In [50]:
the_people = conn.sql("SELECT * FROM people_merge").df()
display(the_people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712721656.665147,d5Oo4EVv6o6RmQ,None
1,2,Person_2,27,City_A,1712721656.665147,w/pw4+vbwsLZxQ,None
2,3,Person_3,28,City_A,1712721656.665147,EbkJ8ltQGEd8ng,None
3,4,Person_4,29,City_A,1712721656.665147,JK7fq9ZZy/w0Qg,None
4,5,Person_5,30,City_A,1712721656.665147,/p2LA8Ap1v3jYA,None


In [51]:
info = generators_pipeline.run(people_2(),
										table_name="people_merge",
										write_disposition="replace",
                               primary_key = "id")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1712721680.7891695 is LOADED and contains no failed jobs


In [54]:
the_people_merge = conn.sql("SELECT * FROM people_merge").df()
display(the_people_merge)

print (f"Sum of Ages {the_people_merge['age'].sum()}")

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,3,Person_3,33,City_B,1712721680.7891695,zOIzEsk4qN161g,Job_3
1,4,Person_4,34,City_B,1712721680.7891695,8mqRBS70ZJ7lTw,Job_4
2,5,Person_5,35,City_B,1712721680.7891695,QjqY+emYjEfyUg,Job_5
3,6,Person_6,36,City_B,1712721680.7891695,yZSLfmZ5chBslg,Job_6
4,7,Person_7,37,City_B,1712721680.7891695,Tje2jFhGkQGeAg,Job_7
5,8,Person_8,38,City_B,1712721680.7891695,ONTdhhtWGEDlWw,Job_8


Sum of Ages 213


# Solution: First make sure that the following modules are installed:

In [57]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [58]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX